In [130]:
#What is the probability that I will get booked if I charge $35/night with 4 repeat guests and 10 reviews?

In [1]:
import pandas as pd
import statsmodels.api as sm
import math
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from os.path import join
from sklearn import metrics

In [2]:
df = pd.read_csv('dog-vacay.csv')

In [3]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
df['fee35'] = df['fee'].map(lambda x: 1 if x >= 35 else 0)

In [5]:
df.head()

,city,fee,names,ratings,repeat,response_time,review,services,boarding,sitting,daycare,checkups,walking,bathing,training,grooming,pickup/dropoff,fee35
0,"Phoenix, Arizona",35,Big Backyard in Central Phoenix,5,8,1,26,2,1,0,1,0,0,0,0,0,0,1
1,"Phoenix, Arizona",35,SUZY Q to the Rescue,5,16,2,30,3,1,0,0,1,0,0,0,0,1,1
2,"Phoenix, Arizona",35,24/7 Dog boarding,0,0,1,0,4,1,0,1,1,1,0,0,0,0,1
3,"Phoenix, Arizona",38,Messy4dogs@gmail.com,5,2,3,3,4,1,0,1,1,1,0,0,0,0,1
4,"Phoenix, Arizona",25,Pet Paradise,5,1,1,5,4,1,0,0,1,1,1,0,0,0,0


In [13]:
cols = ['fee', 'ratings', 'repeat', 'response_time', 'review', 'services']

In [14]:
x = df[cols]
y = df['fee35']

In [15]:
#Define the logistic regression model.
model = sm.Logit(y,x)

In [16]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.395251
         Iterations 7


In [17]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  fee35   No. Observations:                 4662
Model:                          Logit   Df Residuals:                     4656
Method:                           MLE   Df Model:                            5
Date:                Wed, 27 Jan 2016   Pseudo R-squ.:                  0.3793
Time:                        20:55:29   Log-Likelihood:                -1842.7
converged:                       True   LL-Null:                       -2968.8
                                        LLR p-value:                     0.000
=================================================================================
                    coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
fee               0.1727      0.005     31.763      0.000         0.162     0.183
ratings          -0.2599      0.021    -12.571      0.000        -0.300    -0.219
repeat           -0.0096      0.010     -0.976      0.329        -0.029     0.010
response_time    -0.7023      0.053    -13.304      0.000        -0.806    -0.599
review            0.0034      0.005      0.694      0.488        -0.006     0.013
services         -0.9496      0.042    -22.668      0.000        -1.032    -0.867
=================================================================================
"""

In [18]:
def logistic_function(fee, ratings, repeat, response_time, review, services ):
    prob = 1/(1 + math.exp(coeff[0]*repeat+coeff[1]*review))
    return prob

In [19]:
coeff = result.params
print coeff

fee              0.172748
ratings         -0.259922
repeat          -0.009592
response_time   -0.702310
review           0.003372
services        -0.949560
dtype: float64


In [22]:
logistic_function(30, 5, 5, 3, 5, 3)

0.6072742193054314

In [15]:
#SVC TEST

In [16]:
dfTrain, dfTest = train_test_split(df, test_size=0.1, random_state=0)

In [17]:
dfTrain = pd.DataFrame(dfTrain, columns=df.columns)
dfTest = pd.DataFrame(dfTest, columns=df.columns)

In [18]:
x1 = dfTrain[cols]
y1 = dfTrain['fee35']

In [19]:
#Train
clf = SVC(gamma=.001)#can change #

In [20]:
clf.fit(x1,y1)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [21]:
#Test
x2 = dfTest[cols]
y2 = dfTest['fee35']

In [22]:
ypred = clf.predict(x2)

In [23]:
#Converts the array to one with data type np.float64 (i.e. an array where the array elements 
#are the actual floating point values, not pointers to objects)
a = ypred.astype(np.float64)

In [24]:
fpr, tpr, thresholds = metrics.roc_curve(y2, a, pos_label=1)
AUC = metrics.auc(fpr, tpr)

In [25]:
AUC

0.49980773122849625